In [48]:
import kaggle
import zipfile
import os

# Ensure the kaggle.json file is in the correct location
os.environ['KAGGLE_CONFIG_DIR'] = "~/.kaggle/"

# Command to download the dataset
competition = 'playground-series-s4e9'
kaggle.api.competition_download_files(competition, path='.', quiet=False)

# Unzip the downloaded file
zip_file = f'{competition}.zip'
with zipfile.ZipFile(zip_file, 'r') as z:
    z.extractall('.')
    print(f'Extracted all files in {zip_file}')

# Clean up the zip file if desired
os.remove(zip_file)


100%|██████████| 7.84M/7.84M [00:01<00:00, 6.01MB/s]



Extracted all files in playground-series-s4e9.zip


In [23]:
import pandas as pd

In [24]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
dataset = pd.concat([train, test], axis=0).reset_index(drop=True)

dataset.to_csv('dataset.csv', index=False)

## Data preprocessing using Microsoft Data Wrangler

In [25]:
import numpy as np
import re

def clean_data(df):
    # Derive column 'CarAge' from column: 'model_year'
    df.insert(4, "CarAge", 2024 - df["model_year"])
    
    # Drop column: 'model_year'
    df = df.drop(columns=['model_year'])

    # Extract and insert the new columns directly using the insert method
    df.insert(5, 'HP', df['engine'].apply(lambda x: re.search(r'(\d+\.?\d*)\s*HP', x).group(1) if re.search(r'(\d+\.?\d*)\s*HP', x) else None))
    df.insert(6, 'L', df['engine'].apply(lambda x: re.search(r'(\d+\.?\d*)\s*L', x).group(1) if re.search(r'(\d+\.?\d*)\s*L', x) else None))
    
    # Enhanced extraction logic for 'Cylinder'
    df.insert(7, 'Cylinder', df['engine'].apply(lambda x: (
        re.search(r'(\d+)\s*Cylinder', x).group(1) if re.search(r'(\d+)\s*Cylinder', x) 
        else (re.search(r'V(\d+)', x).group(1) if re.search(r'V(\d+)', x) else None))
    ))

    # Convert extracted values to appropriate numeric types
    df['HP'] = pd.to_numeric(df['HP'], errors='coerce')
    df['L'] = pd.to_numeric(df['L'], errors='coerce')
    df['Cylinder'] = pd.to_numeric(df['Cylinder'], errors='coerce')

    # Function to fill NaN values with 0 for electric vehicles
    def fillna_for_electric(df):
        # Check for the presence of the word "Electric" in the 'Engine' column
        electric_condition = df['engine'].str.contains('Electric', case=False, na=False)
        # Fill NaN values in 'HP', 'L', 'Cylinder', and 'Fuel_Type' with 0 where the engine is electric
        df.loc[electric_condition, ['HP', 'L', 'Cylinder', 'fuel_type']] = df.loc[electric_condition, ['HP', 'L', 'Cylinder', 'fuel_type']].fillna(0)
        return df
    
    # Apply the function to fill NaN values for electric vehicles
    df = fillna_for_electric(df)

    # Drop column: 'engine'
    df = df.drop(columns=['engine'])
    df['fuel_type'] = df['fuel_type'].replace(0, 'Electric')

    # Drop column: 'id'
    df = df.drop(columns=['id'])
    
    # Replace missing values with the median of each column in: 'HP'
    df = df.fillna({'HP': df['HP'].median()})

    # Replace missing values with the mean of each column in: 'L'
    df = df.fillna({'L': df['L'].mean().round()})

    # Replace missing values with the mean of each column in: 'Cylinder'
    df = df.fillna({'Cylinder': df['Cylinder'].mean().round()})
    
    # Calculate the overall mode for fuel_type
    overall_mode = df['fuel_type'].mode()[0]

    # Fill missing values with the mode of the same model, or the overall mode if the model mode is not available
    df['fuel_type'] = df.groupby('model')['fuel_type'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else overall_mode))
    
    # Replace missing values with the most common value of each column in: 'accident'
    df = df.fillna({'accident': "missing"})

    # Drop column: 'clean_title'
    df = df.fillna({'clean_title': "missing"})

    return df

df = clean_data(dataset)
df.head()

,brand,model,CarAge,milage,HP,L,Cylinder,fuel_type,transmission,ext_col,int_col,accident,clean_title,price
0,MINI,Cooper S Base,17,213000,172.0,1.6,4.0,Gasoline,A/T,Yellow,Gray,None reported,Yes,4200.0
1,Lincoln,LS V8,22,143250,252.0,3.9,8.0,Gasoline,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999.0
2,Chevrolet,Silverado 2500 LT,22,136731,320.0,5.3,8.0,E85 Flex Fuel,A/T,Blue,Gray,None reported,Yes,13900.0
3,Genesis,G90 5.0 Ultimate,7,19500,420.0,5.0,8.0,Gasoline,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000.0
4,Mercedes-Benz,Metris Base,3,7388,208.0,2.0,4.0,Gasoline,7-Speed A/T,Black,Beige,None reported,Yes,97500.0


In [26]:
df['fuel_type'].isna().sum()

0

In [27]:
df.to_csv('df.csv', index=False)

In [28]:
df["int_col"].unique()

array(['Gray', 'Beige', 'Black', '–', 'Blue', 'White', 'Red', 'Brown',
       'Dark Galvanized', 'Parchment.', 'Boulder', 'Orange',
       'Medium Earth Gray', 'Ebony', 'Canberra Beige', 'Jet Black',
       'Silver', 'Light Platinum / Jet Black', 'Macchiato/Magmagrey',
       'Gold', 'Cloud', 'Rioja Red', 'Global Black', 'Green',
       'Medium Stone', 'Navy Pier', 'Dark Ash', 'BLACK', 'Portland',
       'Sandstone', 'Canberra Beige/Black', 'Diesel Gray / Black',
       'Sarder Brown', 'Black Onyx', 'White / Brown', 'Black/Gun Metal',
       'Slate', 'Satin Black', 'Macchiato Beige/Black', 'Charcoal',
       'Black / Express Red', 'Cappuccino', 'Aragon Brown', 'Parchment',
       'Oyster W/Contrast', 'Adrenaline Red', 'Ebony.', 'Shara Beige',
       'Graystone', 'Pearl Beige', 'Nero Ade', 'Graphite',
       'Tan/Ebony/Ebony', 'Charcoal Black', 'Medium Ash Gray',
       'Ebony Black', 'Light Titanium', 'Sakhir Orange', 'Tan',
       'Rock Gray', 'Brandy', 'Carbon Black', 'Amber',
      

In [29]:
df["ext_col"].unique()

array(['Yellow', 'Silver', 'Blue', 'Black', 'White',
       'Snowflake White Pearl Metallic', 'Gray', 'Green',
       'Santorini Black Metallic', 'Purple', 'Ebony Twilight Metallic',
       'Red', 'Magnetite Black Metallic', 'Diamond Black', 'Vega Blue',
       'Beige', 'Gold', 'Platinum White Pearl', 'Metallic',
       'White Frost Tri-Coat', 'Firecracker Red Clearcoat',
       'Phytonic Blue Metallic', 'Blu', 'Orange', 'Brown',
       'Brilliant Silver Metallic', 'Black Raven', 'Black Clearcoat',
       'Firenze Red', 'Agate Black Metallic', 'Glacial White Pearl',
       'Majestic Plum Metallic', 'designo Diamond White Metallic',
       'Oxford White', 'Black Sapphire Metallic', 'Mythos Black',
       'Granite Crystal Clearcoat Metallic', 'White Diamond Tri-Coat',
       'Magnetite Gray Metallic', 'Carpathian Grey Premium Metallic',
       'designo Diamond White Bright',
       'Phantom Black Pearl Effect / Black Roof', 'Nebula Gray Pearl',
       'Deep Crystal Blue Mica', 'Flame Red

In [30]:
df.dtypes

brand            object
model            object
CarAge            int64
milage            int64
HP              float64
L               float64
Cylinder        float64
fuel_type        object
transmission     object
ext_col          object
int_col          object
accident         object
clean_title      object
price           float64
dtype: object

In [31]:
df['fuel_type'].isna().sum()

0

In [32]:
df['transmission'].unique()

array(['A/T', 'Transmission w/Dual Shift Mode', '7-Speed A/T',
       '8-Speed A/T', '10-Speed Automatic', '1-Speed A/T', '6-Speed A/T',
       '10-Speed A/T', '9-Speed A/T', '8-Speed Automatic',
       '9-Speed Automatic', '5-Speed A/T', 'Automatic',
       '7-Speed Automatic with Auto-Shift', 'CVT Transmission',
       '5-Speed M/T', 'M/T', '6-Speed M/T', '6-Speed Automatic',
       '4-Speed Automatic', '7-Speed M/T', '2-Speed A/T',
       '1-Speed Automatic', 'Automatic CVT', '4-Speed A/T',
       '6-Speed Manual', 'Transmission Overdrive Switch',
       '8-Speed Automatic with Auto-Shift', '7-Speed Manual',
       '7-Speed Automatic', '9-Speed Automatic with Auto-Shift',
       '6-Speed Automatic with Auto-Shift',
       '6-Speed Electronically Controlled Automatic with O', 'F', 'CVT-F',
       '8-Speed Manual', 'Manual', '–', '2', '6 Speed At/Mt',
       '5-Speed Automatic', '2-Speed Automatic', '8-SPEED A/T', '7-Speed',
       'Variable', 'Single-Speed Fixed Gear', '8-SPEED AT',


In [33]:
# Replace en-dash with NaN only in the 'transmission' column
df['transmission'] = df['transmission'].replace("–", np.nan)

transmission_mapping = {
    "a/t": "Automatic",
    "at": "Automatic",
    "automatic": "Automatic",
    "auto-shift": "Automatic",
    "cvt": "CVT",
    "cvt-f": "CVT",
    "m/t": "Manual",
    "mt": "Manual",
    "manual": "Manual",
    "w/dual": "Dual-Clutch",
    "variable": "CVT",
    "f": "F1",
    "overdrive": "Overdrive",
    "electronically": "Electronic",
    "at/mt": "Automatic/Manual",
    "dct": "Dual-Clutch",
}


# Map known transmission types
df['transm_'] = df['transmission'].map(transmission_mapping)

# Define the mapping function
def map_transmission(trans):
    trans = str(trans).lower()
    if trans in ['', '–', '-', 'nan', 'none', 'unknown', 'scheduled for or in production']:
        return 'Unknown'
    elif 'dual-clutch' in trans or 'dct' in trans or 'dual shift' in trans or 'w/dual' in trans:
        return 'Dual-Clutch'
    elif 'cvt' in trans or 'variable' in trans:
        return 'CVT'
    elif 'manual' in trans or 'm/t' in trans or 'mt' in trans:
        return 'Manual'
    elif 'at/mt' in trans:
        return 'Automatic/Manual'
    elif 'automatic' in trans or 'a/t' in trans or 'at' in trans or 'auto' in trans:
        return 'Automatic'
    elif 'single-speed' in trans or 'single speed' in trans:
        return 'Single-Speed'
    elif 'overdrive' in trans:
        return 'Automatic'
    elif 'electronically controlled' in trans or 'electronically' in trans:
        return 'Automatic'
    elif 'f1' in trans or trans.strip() == 'f':
        return 'F1'
    else:
        return 'Unknown'

# Apply the mapping function to unmapped entries
mask = df['transm_'].isnull()
df.loc[mask, 'transm_'] = df.loc[mask, 'transmission'].apply(map_transmission)

# Fill any remaining missing values
df['transm_'].fillna('Unknown', inplace=True)

# Verify the results
print(df['transm_'].unique())


['Automatic' 'Dual-Clutch' 'CVT' 'Manual' 'F1' 'Unknown' 'Single-Speed']


C:\Users\Stefan\AppData\Local\Temp\ipykernel_11212\3112524308.py:58: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['transm_'].fillna('Unknown', inplace=True)


In [34]:
df['fuel_type'].isna().sum()

0

In [35]:
df["transm_"].unique()


array(['Automatic', 'Dual-Clutch', 'CVT', 'Manual', 'F1', 'Unknown',
       'Single-Speed'], dtype=object)

In [36]:
int_replacements = {
    'Medium Earth Gray': 'Gray',
    'Diesel Gray / Black': 'Gray',
    'Dark Ash': 'Gray',
    'Graphite': 'Gray',
    'Dark Galvanized': 'Charcoal',
    'Light Gray': 'Gray',
    'Ash': 'Gray',
    'Jet Black': 'Black',
    'Global Black': 'Black',
    'Black Onyx': 'Black',
    'Parchment.': 'Beige',
    'Sardar Brown': 'Brown',
    'Black/Gun Metal': 'Black',
    'Charcoal Black': 'Charcoal',
    'Ebony': 'Brown',
    'Ebony Black': 'Black',
    'Carbon Black': 'Black',
    'Obsidian Black': 'Black',
    'Black / Saddle Brown': 'Black',
    'Black/Saddle Brown': 'Black',
    'Black / Brown': 'Black',
    'Saddle Brown': 'Brown',
    'Sand Beige': 'Beige',
    'Camel': 'Beige',
    'Parchment': 'Beige',
    'Macchiato Beige/Black': 'Beige',
    'Silk Beige/Espresso Brown': 'Beige',
    'Canberra Beige': 'Beige',
    'Macchiato': 'Beige',
    'Almond Beige': 'Beige',
    'Grace White': 'White',
    'Ivory / Ebony': 'White',
    'Bianco Polar': 'White',
    'White / Brown': 'White',
    'Platinum': 'White',
    'Cloud': 'Blue',
    'Rift Metallic': 'White',
    'Light Platinum / Jet Black': 'Silver',
    'Billet Clearcoat Metallic': 'Silver',
    'Sakhir Orange': 'Orange',
    'Pimento Red w/Ebony': 'Red',
    'Adrenaline Red': 'Red',
    'Rioja Red': 'Red',
    'Classic Red': 'Red',
    'Magma Red': 'Red',
    'Cobalt Blue': 'Blue',
    'Tempest': 'Blue',
    'Stormy Sea': 'Blue',
    'Navy Pier': 'Blue',
    'Charles Blue': 'Blue',
    'Rhapsody Blue': 'Blue',
    'Kyalami Orange': 'Orange',
    'Sakhir Orange/Black': 'Orange',
    'Dark Gray': 'Gray',
    'Deep Garnet': 'Red',
    'Scarlet Ember': 'Red',
    'Beluga': 'Blue',
    'Chestnut': 'Brown',
    'Boulder': 'Gray',
    'Macchiato/Magmagrey': 'Beige',
    'Medium Stone': 'Gray',
    'BLACK': 'Black',
    'Portland': 'Gray',
    'Sandstone': 'Beige',
    'Slate': 'Gray',
    'Cappuccino': 'Brown',
    'Oyster W/Contrast': 'Beige',
    'Nero Ade': 'Black',
    'Light Titanium': 'Silver',
    'Tan': 'Beige',
    'Brandy': 'Brown',
    'Amber': 'Yellow',
    'Hotspur': 'Blue',
    'Chateau': 'Green',
    'Ice': 'Blue',
    'Blk': 'Black',
    'Mesa': 'Brown',
    'Espresso': 'Brown',
    'Ceramic': 'White',
    'Medium Dark Slate': 'Gray',
    'Graphite w/Gun Metal': 'Gray',
    'Cocoa / Dune': 'Brown',
    'Roast': 'Brown',
    'Hotspur Hide': 'Brown',
    'ORANGE': 'Orange',
    'Walnut': 'Brown',
    'Caramel': 'Beige',
    'Giallo Taurus / Nero Ade': 'Yellow',
    'Medium Pewter': 'Gray',
    'Camel Leather': 'Brown',
    'Anthracite': 'Gray',
    'Mocha': 'Brown',
    'Sahara Tan': 'Beige',
    'Porpoise': 'Beige',
    'Deep Cypress': 'Green',
    'Light Slate': 'Gray',
    'Beluga Hide': 'Black',
    'Tupelo': 'Green',
    'Gideon': 'Beige',
    'Medium Light Camel': 'Beige',
    'Nero': 'Black',
    'Deep Chestnut': 'Red',
    'Dark Auburn': 'Brown',
    'Shale': 'Gray',
    'BEIGE': 'Beige',
    'Linen': 'Beige',
    'WHITE': 'White'
}
ext_replacements = {
    'Blu': 'Blue',
    'BLUE': 'Blue',
    'Glacier': 'Blue',
    'BLU ELEOS': 'Blue',
    'Dark Sapphire': 'Navy',
    'Tangerine': 'Orange',
    'Pumpkin': 'Orange',
    'Clementine': 'Orange',
    'Granite': 'Gray',
    'Go Mango!': 'Yellow',
    'Onyx': 'Black',
    'Gecko Pearlcoat': 'Green',
    'Obsidian': 'Black',
    'Metallic': 'Silver',
    'Grigio Nimbus': 'Silver',
    'Chalk': 'White',
    'Bianco Monocerus': 'White',
    'Verde': 'Green',
    'Dark Graphite Metallic': 'Gray',
    'BLACK': 'Black',
    'Dark Moss': 'Green',
    'Granite Crystal Clearcoat Metallic': 'Gray',
    'Ebony Twilight Metallic': 'Black',
    'Satin Steel Metallic': 'Silver',
    'Magnetic Metallic': 'Gray',
    'Dark Matter Metallic': 'Gray',
    'Dark Ash Metallic': 'Gray',
    'Iridium Metallic': 'Gray',
    'Nightfall Mica': 'Navy',
    'Sandstone Metallic': 'Beige',
    'Rift Metallic': 'White',
    'Billet Clearcoat Metallic': 'Silver',
    'Tan': 'Beige',
    'Ice': 'Blue',
    'Hellayella': 'Yellow',
    'Granite': 'Gray',
    'Yulong': 'White',
    'Blueprint': 'Navy',
    'Arancio Borealis': 'Orange',
    'Hellayella Clearcoat': 'Yellow',
    'Moonlight Cloud': 'Navy',
    'Liquid Platinum': 'Silver',
    'Gun Metallic': 'Gray',
    'Manhattan Noir Metallic': 'Gray',
    'Lavender': 'Purple',
    'Violet': 'Purple',
    'Pink': 'Purple',
    'Mauve': 'Plum',
    'Tempest': 'Blue',
    'Nero Daytona': 'Black',
    'Scarlet Ember': 'Red',
    'Infrared Tintcoat': 'Red',
    'Maximum Steel Metallic': 'Gray',
    'Ember Pearlcoat': 'Brown',
    'Rich Garnet Metallic': 'Brown',
    'Tungsten Metallic': 'Gray',
    'Nero Noctis': 'Black',
    'Platinum Quartz Metallic': 'White',
    'Ruby Flare Pearl': 'Red',
    'Bianco Icarus Metallic': 'White',
    'Stormy Sea': 'Blue',
    'Mountain Air Metallic': 'Blue',
    'Wind Chill Pearl': 'White',
    'Iridescent Pearl Tricoat': 'White',
    'Black Cherry': 'Plum',
    'Black Forest Green': 'Green',
    'Maroon': 'Red',
    'Rosso': 'Red',
    'Rosso Corsa': 'Red',
    'Rosso Mars Metallic': 'Red',
    'Quicksilver Metallic': 'Silver',
    'Designo Magno Matte': 'Gray',
    'Granite Crystal Metallic Clearcoat': 'Gray',
    'Bianco Isis': 'White',
    'Sunset Drift Chromaflair': 'Orange',
    'Ametrin Metallic': 'Plum',
    'GT SILVER': 'Silver',
    'Caviar': 'Black'
}

def standardize_and_extract_colors(df, int_replacements, ext_replacements):
    # Step 1: Apply specific color replacements to standardize color names
    df['int_col'] = df['int_col'].replace(int_replacements)
    df['ext_col'] = df['ext_col'].replace(ext_replacements)
    
    # Ensure both columns are lowercase after replacements
    df['int_col'] = df['int_col'].str.lower()
    df['ext_col'] = df['ext_col'].str.lower()

    # Define a list of common base colors
    base_colors = [
        'black', 'white', 'gray', 'silver', 'brown', 'red', 'blue', 'green',
        'beige', 'tan', 'orange', 'yellow', 'purple', 'plum', 'charcoal', 
        'ivory', 'camel', 'chestnut', 'pearl', 'linen', 'graphite', 'copper', 
        'slate', 'bronze', 'sand', 'amber', 'macchiato', 'ebony', 'cocoa', 'navy'
    ]

    # Function to find base color
    def find_base_color(text):
        if isinstance(text, str):
            for color in base_colors:
                if color in text:
                    return color
        return text
    
    # Step 2: Apply base color extraction after standardization
    df['int_col'] = df['int_col'].astype(str).apply(find_base_color)
    df['ext_col'] = df['ext_col'].astype(str).apply(find_base_color)

    return df

# Example usage
df = standardize_and_extract_colors(df, int_replacements, ext_replacements)

In [37]:
def extract_other_features(df):
    
    luxury_brands =  ['Mercedes-Benz', 'BMW', 'Audi', 'Porsche', 'Land', 
                    'Lexus', 'Jaguar', 'Bentley', 'Maserati', 'Lamborghini', 
                    'Rolls-Royce', 'Ferrari', 'McLaren', 'Aston', 'Maybach']
    df['Is_Luxury_Brand'] = df['brand'].apply(lambda x: 1 if x in luxury_brands else 0)
    
    # df.drop(columns = ['brand'] , inplace=True)

    df.drop(columns = ['transmission'], inplace=True)

     # Avoid division by zero
    df['Mileage_per_Year'] = np.where(
        df['CarAge'] == 0,
        df['milage'],
        df['milage'] / df['CarAge']
    )

    df['milage_with_age'] =  df.groupby('CarAge')['milage'].transform('mean')
    
    df['Mileage_per_Year_with_age'] =  df.groupby('CarAge')['Mileage_per_Year'].transform('mean')

    return df

df = extract_other_features(df)

In [38]:
df.head()

,brand,model,CarAge,milage,HP,L,Cylinder,fuel_type,ext_col,int_col,accident,clean_title,price,transm_,Is_Luxury_Brand,Mileage_per_Year,milage_with_age,Mileage_per_Year_with_age
0,MINI,Cooper S Base,17,213000,172.0,1.6,4.0,Gasoline,yellow,gray,None reported,Yes,4200.0,Automatic,0,12529.411765,119070.619650,7004.154097
1,Lincoln,LS V8,22,143250,252.0,3.9,8.0,Gasoline,silver,beige,At least 1 accident or damage reported,Yes,4999.0,Automatic,0,6511.363636,119616.498612,5437.113573
2,Chevrolet,Silverado 2500 LT,22,136731,320.0,5.3,8.0,E85 Flex Fuel,blue,gray,None reported,Yes,13900.0,Automatic,0,6215.045455,119616.498612,5437.113573
3,Genesis,G90 5.0 Ultimate,7,19500,420.0,5.0,8.0,Gasoline,black,black,None reported,Yes,45000.0,Dual-Clutch,0,2785.714286,68016.149142,9716.592735
4,Mercedes-Benz,Metris Base,3,7388,208.0,2.0,4.0,Gasoline,black,beige,None reported,Yes,97500.0,Automatic,1,2462.666667,29210.023980,9736.674660


In [39]:
train = df[df['price'].notnull()]
test = df[df['price'].isnull()]

In [40]:
import numpy as np
import pandas as pd

import lightgbm as lgb

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Set random state for reproducibility
rs = 42


In [41]:
# from sklearn.preprocessing import StandardScaler

X = train.drop(columns=['price'])
y = train['price']

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=rs)

import numpy as np

# print("NaNs in X_train:", X.isnull().sum())
# print("NaNs in y_train:", y.isnull().sum())

# Check for infinite values in numerical columns
# numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
# print("Infinite values in X_train:", np.isinf(X[numerical_cols]).sum())


categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
print("Categorical columns:", categorical_cols)


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ],
    remainder='passthrough'  # Keep the remaining columns as is
)

def create_pipeline(model):
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    return pipeline

lgbm_space = {
    'num_boost_round': scope.int(hp.quniform('num_boost_round', 100, 2500, 1)),  
    'num_leaves': scope.int(hp.quniform('num_leaves', 20, 600, 1)),  
    'learning_rate': hp.loguniform('learning_rate', -9.21, -1.01), 
    'feature_fraction': hp.uniform('feature_fraction', 0.3, 1),  
    'random_state': rs,
}


Categorical columns: ['brand', 'model', 'fuel_type', 'ext_col', 'int_col', 'accident', 'clean_title', 'transm_']


In [42]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from hyperopt import STATUS_OK

# Objective function for hyperparameter optimization
def objective(params, model_type='lgbm'):
    if model_type == 'lgbm':
        model = lgb.LGBMRegressor(
            num_leaves=int(params['num_leaves']),
            learning_rate=params['learning_rate'],
            feature_fraction=params['feature_fraction'],
            random_state=params['random_state'],
            n_estimators=int(params['num_boost_round'])
        )
    else:
        raise ValueError("Unsupported model type")

    # Create a pipeline with the model
    pipeline = create_pipeline(model)

    # Perform cross-validation
    neg_mse = cross_val_score(
        pipeline, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1
    ).mean()
    rmse = np.sqrt(-neg_mse)
    return {'loss': rmse, 'status': STATUS_OK}


In [43]:
def optimize(space, model_type, max_evals=50):
    trials = Trials()
    best = fmin(
        fn=lambda params: objective(params, model_type=model_type),
        space=space,
        algo=tpe.suggest,
        max_evals=max_evals,
        trials=trials,
        rstate=np.random.default_rng(rs)
    )
    return best, trials


In [44]:
print("Optimizing LightGBM...")
best_lgbm_params, trials_lgbm = optimize(lgbm_space, 'lgbm', max_evals=50)
print("Best LightGBM params:", best_lgbm_params)


Optimizing LightGBM...
100%|██████████| 50/50 [1:42:37<00:00, 123.14s/trial, best loss: 74156.08203022127]  
Best LightGBM params: {'feature_fraction': 0.410025790393328, 'learning_rate': 0.015840568239058373, 'num_boost_round': 277.0, 'num_leaves': 105.0}


In [45]:
best_lgbm = lgb.LGBMRegressor(
    num_leaves=int(best_lgbm_params['num_leaves']),
    learning_rate=best_lgbm_params['learning_rate'],
    feature_fraction=best_lgbm_params['feature_fraction'],
    random_state=rs,
    n_estimators=int(best_lgbm_params['num_boost_round'])
)

pipeline_lgbm = create_pipeline(best_lgbm)
pipeline_lgbm.fit(X_train, y_train)
preds_lgbm = pipeline_lgbm.predict(X_val)
preds_lgbm = preds_lgbm
y_val = y_val

rmse_lgbm = np.sqrt(mean_squared_error(y_val, preds_lgbm))
print(f"\nLightGBM RMSE on validation set: {rmse_lgbm}")


[LightGBM] [Warning] feature_fraction is set=0.410025790393328, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.410025790393328
[LightGBM] [Warning] feature_fraction is set=0.410025790393328, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.410025790393328
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005632 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3755
[LightGBM] [Info] Number of data points in the train set: 150826, number of used features: 1434
[LightGBM] [Info] Start training from score 43890.785316
[LightGBM] [Warning] feature_fraction is set=0.410025790393328, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.410025790393328

LightGBM RMSE on validation set: 68090.25348329682


In [46]:
# Predictions on the test set
X_test = test.drop(columns=['price'])
y_test = test['price']

############################################
pipeline_lgbm.fit(X, y)
preds_test_lgbm = np.expm1(pipeline_lgbm.predict(X_test))



[LightGBM] [Warning] feature_fraction is set=0.410025790393328, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.410025790393328
[LightGBM] [Warning] feature_fraction is set=0.410025790393328, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.410025790393328
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006324 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4091
[LightGBM] [Info] Number of data points in the train set: 188533, number of used features: 1602
[LightGBM] [Info] Start training from score 43878.016178
[LightGBM] [Warning] feature_fraction is set=0.410025790393328, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.410025790393328


C:\Users\Stefan\AppData\Local\Temp\ipykernel_11212\719877758.py:7: RuntimeWarning: overflow encountered in expm1
  preds_test_lgbm = np.expm1(pipeline_lgbm.predict(X_test))


In [47]:
output = pd.DataFrame({'id': test.index, 'price': preds_test_lgbm})
output.to_csv('submission.csv', index=False)